<a href="https://colab.research.google.com/github/MohammedHajjey/Simple_Information_Retrieval_Project/blob/main/informationRetrievalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mateibejan/15000-gutenberg-books")
print("Path to dataset files:", path)

100%|██████████| 345k/345k [00:00<00:00, 41.9MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/mateibejan/15000-gutenberg-books/versions/5


In [2]:
import shutil
destenation_path = '/content'
shutil.move(path, destenation_path)

'/content/5'

In [3]:
!pip install -U spacy
!pip install -U nltk

# English laguage model
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import numpy as np
import pandas as pd
import math
import os
import re
import string

import spacy
nlp = spacy.load('en_core_web_sm')

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('popular')
# NOT: stemmimng dependencies

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import defaultdict

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzippin

In [6]:
df = pd.read_csv('/content/5/gutenberg_metadata.csv')
df.head(10)

,Title,Author,Link,Bookshelf
0,The Extermination of the American Bison,William T. Hornaday,http://www.gutenberg.org/ebooks/17748,Animal
1,Deadfalls and Snares,A. R. Harding,http://www.gutenberg.org/ebooks/34110,Animal
2,Artistic Anatomy of Animals,Édouard Cuyer,http://www.gutenberg.org/ebooks/38315,Animal
3,"Birds, Illustrated","Color Photography, Vol. 1, No. 1 Various",http://www.gutenberg.org/ebooks/30221,Animal
4,On Snake-Poison: Its Action and Its Antidote,A. Mueller,http://www.gutenberg.org/ebooks/32947,Animal
5,Fifty Years a Hunter and Trapper,E. N. Woodcock,http://www.gutenberg.org/ebooks/34063,Animal
6,What Bird is That?,Frank M. Chapman,http://www.gutenberg.org/ebooks/31751,Animal
7,Fox Trapping: A Book of Instruction Telling Ho...,NaN,http://www.gutenberg.org/ebooks/34076,Animal
8,A Guide for the Study of Animals,"Lucas, Shinn, Smallwood, and Whitney",http://www.gutenberg.org/ebooks/34984,Animal
9,Our Vanishing Wild Life: Its Extermination and...,William T. Hornaday,http://www.gutenberg.org/ebooks/13249,Animal


In [7]:
def preprocess_content(text):

    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))

    doc = nlp(text)
    tokens = [token.text for token in doc]

    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()

    # Filter: stopwords, letters, numbers
    filtered_tokens = []
    for word in tokens:
        if word in stop_words:
            continue
        if word.isnumeric():
            continue
        if len(word) == 1:
            continue
        stemmed = stemmer.stem(word)
        filtered_tokens.append(stemmed)

    return filtered_tokens



Combining Columns

In [8]:
def prepare_dataset(df):
    df['combined_text'] = df['Title'].fillna('') + ' ' + df['Author'].fillna('') + ' ' + df['Bookshelf'].fillna('')
    df['processed_tokens'] = df['combined_text'].apply(preprocess_content)
    return df


test

In [9]:
df = prepare_dataset(df)
df.head()

,Title,Author,Link,Bookshelf,combined_text,processed_tokens
0,The Extermination of the American Bison,William T. Hornaday,http://www.gutenberg.org/ebooks/17748,Animal,The Extermination of the American Bison Willia...,"[extermin, american, bison, william, hornaday,..."
1,Deadfalls and Snares,A. R. Harding,http://www.gutenberg.org/ebooks/34110,Animal,Deadfalls and Snares A. R. Harding Animal,"[deadfal, snare, hard, anim]"
2,Artistic Anatomy of Animals,Édouard Cuyer,http://www.gutenberg.org/ebooks/38315,Animal,Artistic Anatomy of Animals Édouard Cuyer Animal,"[artist, anatomi, anim, édouard, cuyer, anim]"
3,"Birds, Illustrated","Color Photography, Vol. 1, No. 1 Various",http://www.gutenberg.org/ebooks/30221,Animal,"Birds, Illustrated Color Photography, Vol. 1, ...","[bird, illustr, color, photographi, vol, vario..."
4,On Snake-Poison: Its Action and Its Antidote,A. Mueller,http://www.gutenberg.org/ebooks/32947,Animal,On Snake-Poison: Its Action and Its Antidote A...,"[snakepoison, action, antidot, mueller, anim]"


Building inverted index

In [10]:
def build_inverted_index(df):

    inverted_index = defaultdict(lambda: {'postings': [], 'total_frequency': 0})

    for doc_id, row in df.iterrows():
        term_freq = defaultdict(int)
        for token in row['processed_tokens']:
            term_freq[token] += 1

        for term, freq in term_freq.items():
            inverted_index[term]['postings'].append({'docId': doc_id, 'frequency': freq})
            inverted_index[term]['total_frequency'] += freq

    for term in inverted_index:
        inverted_index[term]['document_frequency'] = len(inverted_index[term]['postings'])

    return dict(inverted_index)

Getting posting list of inverted inedex

In [11]:
def display_posting_list(inverted_index):
    """
    Sadece alfabetik karakterlerden oluşan terimler için
    doc_freq ve postings listesi döner.
    """
    rows = []

    for term in sorted(inverted_index.keys()):
        # take the terms as words (a-z / A-Z)
        if not re.fullmatch(r'^[a-zA-Z]{2,}$', term):
            continue  #if it is contain any  numeric content then skip

        entry = inverted_index[term]
        doc_ids = sorted(post['docId'] for post in entry['postings']) if 'postings' in entry else []

        rows.append({
            'term': term,
            'doc_freq': entry.get('document_frequency', len(doc_ids)),
            'postings_list': doc_ids
        })

    return pd.DataFrame(rows)


In [13]:
inverted_index = build_inverted_index(df)

posting_df = display_posting_list(inverted_index)
print(posting_df.head(20))

            term  doc_freq                                      postings_list
0           aard         4                         [9085, 9480, 12562, 12687]
1          aaron         4                           [5567, 6251, 7422, 7975]
2      aaronsohn         1                                             [2680]
3            abb         1                                             [1689]
4           abba         1                                             [1949]
5          abbey         8  [1329, 1346, 1370, 1377, 5762, 6446, 9999, 10306]
6           abbi         5                      [112, 3067, 3183, 4038, 4154]
7          abbot         8   [2669, 3177, 3412, 4148, 4383, 6025, 6709, 8616]
8         abbott        52  [398, 742, 769, 1307, 2493, 2510, 2786, 3229, ...
9            abc         7         [86, 2736, 3707, 5490, 6174, 10601, 10843]
10           abd         1                                             [1970]
11         abdic         1                                      

Merge Algorithm

In [14]:
def merge_postings(list1, list2):

    result = []
    i, j = 0, 0
    while i < len(list1) and j < len(list2):
        if list1[i] == list2[j]:
            result.append(list1[i])
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            i += 1
        else:
            j += 1
    return result

def optimized_boolean_and(query, inverted_index):
    """
    query: the text that writen by user (for exmple: 'caesar brutus')
    inverted_index: returned by build_inverted_index function

    Geriye: kesişim sonucu docId listesi döner
    """
    #  Query normalization ( preprocess_content)
    query_tokens = preprocess_content(query)

    if not query_tokens:
        return []

    postings_lists = []
    for term in query_tokens:
        if term in inverted_index:
            postings = sorted(post['docId'] for post in inverted_index[term]['postings'])
            postings_lists.append(postings)
        else:
            return []  # empty text
    #  Optimize: start to merge it using posting list
    postings_lists.sort(key=len)
    result = postings_lists[0]

    for i in range(1, len(postings_lists)):
        result = merge_postings(result, postings_lists[i])

    return result


Display tokens of documents

In [18]:
def generate_token_sequence(df):
    """
    Returns a list of tuples (token, doc_id)
    """
    token_sequence = []

    for doc_id, row in df.iterrows():
        tokens = row['processed_tokens']
        for token in tokens:
            token_sequence.append((token, doc_id))

    return token_sequence
token_sequence = generate_token_sequence(df)
token_df = pd.DataFrame(token_sequence, columns=['Term', 'docID'])
print(token_df.head(20))

           Term  docID
0      extermin      0
1      american      0
2         bison      0
3       william      0
4      hornaday      0
5          anim      0
6       deadfal      1
7         snare      1
8          hard      1
9          anim      1
10       artist      2
11      anatomi      2
12         anim      2
13      édouard      2
14        cuyer      2
15         anim      2
16         bird      3
17      illustr      3
18        color      3
19  photographi      3


Display Boolean Matrix

In [19]:
def build_boolean_matrix(token_sequence, num_docs):

   # using term-document pairs create Boolean matrix Token-DocID.

    matrix_dict = defaultdict(lambda: [0] * num_docs)

    for token, doc_id in token_sequence:
        matrix_dict[token][doc_id] = 1  # sadece geçtiyse 1

    # DataFrame
    columns = [f'Doc{doc_id}' for doc_id in range(num_docs)]
    boolean_matrix = pd.DataFrame.from_dict(matrix_dict, orient='index', columns=columns)

    boolean_matrix.index.name = 'Term'
    return boolean_matrix.reset_index()

boolean_matrix = build_boolean_matrix(token_sequence, len(df))

# first 20 terms
print(boolean_matrix.head(20))


           Term  Doc0  Doc1  Doc2  Doc3  Doc4  Doc5  Doc6  Doc7  Doc8  ...  \
0      extermin     1     0     0     0     0     0     0     0     0  ...   
1      american     1     0     0     0     0     0     0     0     0  ...   
2         bison     1     0     0     0     0     0     0     0     0  ...   
3       william     1     0     0     0     0     0     0     0     0  ...   
4      hornaday     1     0     0     0     0     0     0     0     0  ...   
5          anim     1     1     1     1     1     1     1     1     1  ...   
6       deadfal     0     1     0     0     0     0     0     0     0  ...   
7         snare     0     1     0     0     0     0     0     1     0  ...   
8          hard     0     1     0     0     0     0     0     0     0  ...   
9        artist     0     0     1     0     0     0     0     0     0  ...   
10      anatomi     0     0     1     0     0     0     0     0     0  ...   
11      édouard     0     0     1     0     0     0     0     0 

Calculating tf_idf

In [20]:
def compute_tf(df):
    tf = defaultdict(lambda: defaultdict(int))  # {doc_id: {term: freq}}

    for doc_id, row in df.iterrows():
        for term in row['processed_tokens']:
            tf[doc_id][term] += 1

    return tf

def compute_df(tf):
    df_dict = defaultdict(set)  # {term: set(doc_ids)}

    for doc_id in tf:
        for term in tf[doc_id]:
            df_dict[term].add(doc_id)

    return {term: len(doc_ids) for term, doc_ids in df_dict.items()}


def compute_idf(df_dict, N):
    idf = {}
    for term, df in df_dict.items():
        idf[term] = math.log(N / df) if df else 0
    return idf


def compute_tf_idf(tf, idf):
    tf_idf = defaultdict(dict)  # {doc_id: {term: tfidf}}

    for doc_id in tf:
        for term in tf[doc_id]:
            tf_idf[doc_id][term] = tf[doc_id][term] * idf.get(term, 0)

    return tf_idf


In [21]:
def compute_query_vector(query_terms, idf):
    from collections import Counter

    term_freq = Counter(query_terms)
    query_vector = {}

    for term, tf in term_freq.items():
        query_vector[term] = tf * idf.get(term, 0)

    return query_vector


Calculating Cosine Similarity for document ranking

In [22]:
def cosine_similarity(vec1, vec2):
    common = set(vec1) & set(vec2)
    dot = sum(vec1[t] * vec2[t] for t in common)
    norm1 = math.sqrt(sum(v ** 2 for v in vec1.values()))
    norm2 = math.sqrt(sum(v ** 2 for v in vec2.values()))
    return dot / (norm1 * norm2) if norm1 and norm2 else 0.0


In [23]:
def rank_documents(query, tf_idf_vectors, idf):
    query_tokens = preprocess_content(query)
    query_vector = compute_query_vector(query_tokens, idf)

    scores = []
    for doc_id, doc_vector in tf_idf_vectors.items():
        sim = cosine_similarity(query_vector, doc_vector)
        if sim > 0:
            scores.append((doc_id, sim))

    return sorted(scores, key=lambda x: x[1], reverse=True)


Evaluation of relevant

In [24]:
def evaluate_retrieval(retrieved_docs, relevant_docs):
    """
    Precision, Recall ve F1 Score calculation.

    Parameters:
        retrieved_docs: List of document IDs returned by the system
        relevant_docs : ground truth document ID list

    Returns:
        precision, recall, f1_score (float)
    """
    retrieved = set(retrieved_docs)
    relevant = set(relevant_docs)

    tp = len(retrieved & relevant)  # doğru pozitif
    fp = len(retrieved - relevant)  # yanlış pozitif
    fn = len(relevant - retrieved)  # eksik (bulunamayan doğrular)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1_score  = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    return precision, recall, f1_score


Calculation for multiple queries

In [25]:
def average_precision(retrieved, relevant):
    """
    Sıralı retrieval sonuçlarına göre Average Precision (AP) hesaplar.

    Parameters:
        retrieved: sıralı dönen belge ID listesi
        relevant : ground truth belgeler

    Returns:
        AP (float)
    """
    relevant = set(relevant)
    hits = 0
    sum_precisions = 0.0

    for i, doc_id in enumerate(retrieved):
        if doc_id in relevant:
            hits += 1
            precision_at_i = hits / (i + 1)
            sum_precisions += precision_at_i

    return sum_precisions / len(relevant) if relevant else 0.0
def mean_average_precision(all_queries_results):
    """
    Calculates average AP (MAP) for multiple queries.
    Parameters:
        all_queries_results:
            {
            query1: {'retrieved': [...], 'relevant': [...]},
            }
    Returns:MAP (float)
    """
    ap_scores = []
    for q in all_queries_results:
        ap = average_precision(all_queries_results[q]['retrieved'], all_queries_results[q]['relevant'])
        ap_scores.append(ap)

    return sum(ap_scores) / len(ap_scores) if ap_scores else 0.0


Finding relevant documents

In [68]:
# It takes the tokens passed in the query and returns the documents containing these tokens.
def find_relevant_docs(query, df):
    query_tokens = preprocess_content(query)
    relevant_docs = []

    for i, row in df.iterrows():
        doc_tokens = set(row['processed_tokens'])
        if all(token in doc_tokens for token in query_tokens):
            relevant_docs.append(i)

    return relevant_docs


User query section

In [80]:
# TF-IDF
tf = compute_tf(df)
df_dict = compute_df(tf)
idf = compute_idf(df_dict, len(df))
tf_idf_vectors = compute_tf_idf(tf, idf)

query = input("Lütfen sorgunuzu girin: ")

results = rank_documents(query, tf_idf_vectors, idf)

retrieved_docs = [doc_id for doc_id, _ in results[:10]]

relevant_docs = find_relevant_docs(query, df)

# Top 10
print("\n En İyi 10 Sonuç:\n")
for rank, (doc_id, score) in enumerate(results[:10], 1):
    title = df.loc[doc_id, 'Title']
    link = df.loc[doc_id, 'Link']
    print(f"{rank}. Doc {doc_id} → {title}")
    print(f"   Link : {link}")
    print(f"   Score: {score:.6f}")
    print("-" * 50)

#  Evaluation
if relevant_docs:
    precision, recall, f1 = evaluate_retrieval(retrieved_docs, relevant_docs)
    ap = average_precision(retrieved_docs, relevant_docs)

    print(f"\n Değerlendirme (TOP 10 üzerinden):")
    print(f"Precision : {precision:.6f}")
    print(f"Recall    : {recall:.6f}")
    print(f"F1-Score  : {f1:.6f}")
    print(f"AP        : {ap:.6f}")
else:
    print(" Bu sorguyla eşleşen herhangi bir ilgili belge bulunamadı.")


Lütfen sorgunuzu girin: wild animal home

🔝 En İyi 10 Sonuç:

1. Doc 17 → Wild Animals at Home
   Link : http://www.gutenberg.org/ebooks/27887
   Score: 0.710572
--------------------------------------------------
2. Doc 58 → Wild Animals at Home
   Link : http://www.gutenberg.org/ebooks/27887
   Score: 0.586140
--------------------------------------------------
3. Doc 15034 → Home Again, Home Again
   Link : http://www.gutenberg.org/ebooks/17138
   Score: 0.453924
--------------------------------------------------
4. Doc 15059 → Home Again, Home Again
   Link : http://www.gutenberg.org/ebooks/17138
   Score: 0.453924
--------------------------------------------------
5. Doc 380 → Wild Animals I Have Known
   Link : http://www.gutenberg.org/ebooks/3031
   Score: 0.378506
--------------------------------------------------
6. Doc 207 → Wild Animals I Have Known
   Link : http://www.gutenberg.org/ebooks/3031
   Score: 0.367829
--------------------------------------------------
7. Doc 2739 